In [268]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import functions as fun
from linearmodels.iv import IV2SLS
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

In [269]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
9,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
10,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
21,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
28,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
29,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
30,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
31,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
32,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
41,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [270]:
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,1.037e-02,Aiways,U5,400,2.846e+05,201,34,SUV,C,CN,257,1.0
1,1,2022,5.976e-03,Aiways,U5,400,3.137e+05,201,34,SUV,C,CN,183,1.0
2,1,2023,2.860e-03,Aiways,U5,400,2.645e+05,201,34,SUV,C,CN,177,1.0
3,2,2023,4.848e-05,Aiways,U6,405,3.606e+05,214,34,SUV,C,CN,3,1.0
4,3,2019,4.063e-02,Audi,e-tron,375,9.797e+05,402,17,SUV,F,DE,222,1.0
5,3,2020,3.468e-02,Audi,e-tron,375,8.901e+05,402,17,SUV,F,DE,491,1.0
6,3,2021,1.049e-02,Audi,e-tron,375,8.000e+05,402,17,SUV,F,DE,260,1.0
7,3,2022,1.757e-02,Audi,e-tron,375,7.897e+05,402,17,SUV,F,DE,538,1.0
8,3,2023,1.099e-03,Audi,e-tron,375,6.730e+05,402,17,SUV,F,DE,68,1.0
9,4,2021,3.391e-03,Audi,e-tron GT,472,1.279e+06,522,17,Sedan,F,DE,84,1.0


In [271]:
def regions(country):
    if country in ['DE', 'FR', 'DK', 'IT', 'RO', 'CZ', 'ES', 'SE']:
        return 'EU'
    elif country in ['JP', 'KR']:
        return 'Asia'
    elif country in ['CN']:
        return 'China'
    else:
        return country
    
data['Region'] = data['Country'].apply(regions)

In [272]:
fun.BLP(data, 'Range')
fun.BLP(data, 'HP')
fun.BLP(data, 'Chargetime')
fun.GH(data, 'Range', 0.5)
fun.GH(data, 'HP', 0.5)
fun.GH(data, 'Chargetime', 0.5)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,Region,Range_BLP,HP_BLP,Chargetime_BLP,Range_GH,HP_GH,Chargetime_GH
0,1,2021,1.037e-02,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0,China,21513,13583,1861,14999,7405,1317
1,1,2022,5.976e-03,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0,China,32180,20339,2664,21535,10602,1796
2,1,2023,2.860e-03,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0,China,41774,26403,3241,27923,15010,2099
3,2,2023,4.848e-05,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0,China,41769,26390,3241,27468,14562,2099
4,3,2019,4.063e-02,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,EU,5809,3901,570,4709,3105,533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,189,2023,6.464e-05,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4,1.0,EU,41699,26336,3247,21894,17776,1813
330,190,2021,1.465e-02,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363,1.0,EU,21456,13382,1867,11699,10265,1138
331,190,2022,3.331e-02,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020,1.0,EU,32123,20138,2670,17028,16217,1483
332,190,2023,3.175e-02,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965,1.0,EU,41717,26202,3247,22067,19786,1813


In [273]:
product_data = data.rename(columns={
    'Year': 'market_ids',
    'Model': 'product_ids',
    'Market_share': 'shares',
    'Price': 'prices',
    'Manufacturer': 'firm_ids',
})

In [274]:
#Scale for better intepretation
product_data['prices'] = product_data['prices']/10_000 #(Change in ms(%) for change in pris in 10.000)
product_data['HP'] = product_data['HP']/10           #(Change in ms(%) for change in HP in 10)
product_data['Range'] = product_data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [275]:
# Creating dummy for china
product_data['China'] = (product_data['Country'] == 'CN').astype(int)

# Outside share

In [276]:
product_data.loc[product_data['market_ids'] == 2013, 'shares'] = product_data.loc[product_data['market_ids'] == 2013, 'Sales'] / 180632
product_data.loc[product_data['market_ids'] == 2014, 'shares'] = product_data.loc[product_data['market_ids'] == 2014, 'Sales'] / 188406
product_data.loc[product_data['market_ids'] == 2015, 'shares'] = product_data.loc[product_data['market_ids'] == 2015, 'Sales'] / 206653
product_data.loc[product_data['market_ids'] == 2016, 'shares'] = product_data.loc[product_data['market_ids'] == 2016, 'Sales'] / 222471
product_data.loc[product_data['market_ids'] == 2017, 'shares'] = product_data.loc[product_data['market_ids'] == 2017, 'Sales'] / 221471
product_data.loc[product_data['market_ids'] == 2018, 'shares'] = product_data.loc[product_data['market_ids'] == 2018, 'Sales'] / 252328
product_data.loc[product_data['market_ids'] == 2019, 'shares'] = product_data.loc[product_data['market_ids'] == 2019, 'Sales'] / 258727
product_data.loc[product_data['market_ids'] == 2020, 'shares'] = product_data.loc[product_data['market_ids'] == 2020, 'Sales'] / 230060
product_data.loc[product_data['market_ids'] == 2021, 'shares'] = product_data.loc[product_data['market_ids'] == 2021, 'Sales'] / 222210
product_data.loc[product_data['market_ids'] == 2022, 'shares'] = product_data.loc[product_data['market_ids'] == 2022, 'Sales'] / 181030
product_data.loc[product_data['market_ids'] == 2023, 'shares'] = product_data.loc[product_data['market_ids'] == 2023, 'Sales'] / 203690

In [277]:
product_data['outside_share'] = 1 - product_data.groupby('market_ids')['shares'].transform('sum')
product_data[['shares', 'outside_share']].describe()

,shares,outside_share
count,3.340e+02,334.000
mean,2.128e-03,0.841
std,5.618e-03,0.109
min,4.347e-06,0.696
25%,1.227e-04,0.696
50%,7.149e-04,0.831
75%,2.161e-03,0.938
max,8.814e-02,0.998


# Pure Logit OLS Demand

In [278]:
product_data['demand_instruments0'] = product_data['prices']
ols = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)')
ols_problem = pyblp.Problem(ols, product_data)
ols_results = ols_problem.solve(se_type='robust')
ols_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +2.06E-28     0        +1.84E+01          +1.86E+01    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices        Range         HP       Chargetime 
-----------  -----------  -----------  -----------
 -4.53E-02    +1.37E-01    -4.58E-03    -3.14E-02 
(+6.16E-03)  (+1.69E-02)  (+1.50E-02)  (+1.27E-02)

In [279]:
total_count = 0
for i in range(2013,2024):
    ols_elasticities = ols_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(ols_elasticities).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 31


# Pure Logit BLP Demand

In [280]:
product_data['demand_instruments0'] = product_data['Range_BLP']
product_data['demand_instruments1'] = product_data['HP_BLP']
product_data['demand_instruments2'] = product_data['Chargetime_BLP']
blp = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)')
blp_problem = pyblp.Problem(blp, product_data)
blp_results = blp_problem.solve(se_type='robust')
blp_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +7.46E+00     0        +1.82E+07          +1.76E+02    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices        Range         HP       Chargetime 
-----------  -----------  -----------  -----------
 -2.20E-02    +1.40E-01    -4.45E-02    -3.43E-02 
(+4.28E-02)  (+3.23E-02)  (+5.64E-02)  (+1.42E-02)

In [281]:
total_count = 0
for i in range(2013,2024):
    blp_elasticities = blp_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(blp_elasticities).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 229


# Pure Logit GH Demand

In [282]:
product_data['demand_instruments0'] = product_data['Range_GH']
product_data['demand_instruments1'] = product_data['HP_GH']
product_data['demand_instruments2'] = product_data['Chargetime_GH']
gh = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)+C(market_ids)')
gh_problem = pyblp.Problem(gh, product_data)
gh_results = gh_problem.solve(se_type='robust')
gh_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +2.94E+00     0        +1.11E+06          +3.90E+01    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices        Range         HP       Chargetime 
-----------  -----------  -----------  -----------
 -1.12E-01    +1.65E-01    +8.09E-02    -1.90E-02 
(+2.71E-02)  (+2.39E-02)  (+3.42E-02)  (+1.25E-02)

In [283]:
total_count = 0
for i in range(2013,2024):
    gh_elasticities = gh_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(gh_elasticities).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 0


# Nested Logit GH Demand

In [284]:
def nested_logit(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['demand_instruments20'] = groups['shares'].transform(np.size)
    nl_formulations = pyblp.Formulation('1 + prices + Range + HP + Chargetime', absorb='C(firm_ids)+C(market_ids)')
    problem = pyblp.Problem(nl_formulations, df)
    return problem.solve(rho=0.5, se_type='robust') # rho is initial value at which the optimization routine will start

In [285]:
product_data['nesting_ids'] = product_data['Segment']
nl_results = nested_logit(product_data)
nl_results

Problem Results Summary:
GMM   Objective    Projected     Reduced   Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm   Hessian   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------  -------  ----------------  -----------------
 2    +5.52E+00    +5.52E-12    +1.40E+01     0        +1.08E+06          +5.55E+03    

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:00       Yes          2             8     

Rho Estimates (Robust SEs in Parentheses):
All Groups 
-----------
 +3.44E-01 
(+3.88E-01)

Beta Estimates (Robust SEs in Parentheses):
  prices        Range         HP       Chargetime 
-----------  -----------  -----------  -----------
 -8.87E-02    +1.37E-01    +5.65E-02    -2.50E-02 
(+3.03E-02)  (+3.19E-02)  (+3.38E-02)  (+1.21E-02)

In [286]:
data2023 = product_data[product_data['market_ids'] == 2023]
nl_elasticities=  pd.DataFrame(nl_results.compute_elasticities(market_id=2023, name='prices'), index = data2023['product_ids'], columns=data2023['product_ids'])
nl_elasticities

product_ids,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,iX,iX1,iX3,Atto 3,Dolphin,Han,Seal,Tang,Berlingo,C4,ë-C4 X,e-SpaceTourer,Jumpy,Born,Spring,...,Taycan,Megane,Zoe,Enyaq iV,Fortwo,Korando,Solterra,Model 3,Model S,Model X,Model Y,bZ4X,Proace City Verso,Proace Verso,Golf,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
product_ids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
U5,-3.565,2.565e-04,0.002,0.001,0.367,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,0.053,6.416e-05,0.205,1.759e-04
U6,0.011,-4.875e+00,0.002,0.001,0.367,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,0.053,6.416e-05,0.205,1.759e-04
e-tron,0.002,4.713e-05,-8.817,0.173,0.067,0.016,1.380e-05,0.042,0.012,0.482,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,6.234e-04,...,1.200,0.017,3.320e-04,0.058,2.750e-05,4.971e-05,8.234e-04,0.066,0.780,0.459,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,6.017e-04,0.010,6.416e-05,0.038,1.759e-04
e-tron GT,0.002,4.713e-05,0.282,-16.312,0.067,0.016,1.380e-05,0.042,0.012,0.482,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,6.234e-04,...,1.200,0.017,3.320e-04,0.058,2.750e-05,4.971e-05,8.234e-04,0.066,0.780,0.459,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,6.017e-04,0.010,6.416e-05,0.038,1.759e-04
Q4 e-tron,0.011,2.565e-04,0.002,0.001,-8.536,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,0.053,6.416e-05,0.205,1.759e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.002,4.713e-05,0.002,0.001,0.067,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,3.660e-01,...,0.008,0.017,3.320e-04,0.058,1.615e-02,4.971e-05,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,-2.011e+00,0.010,6.416e-05,0.038,1.759e-04
C40,0.011,2.565e-04,0.002,0.001,0.367,0.016,1.380e-05,0.042,0.012,0.003,0.017,0.112,0.02,0.016,0.011,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.015,0.013,8.063e-04,1.247e-04,0.103,6.234e-04,...,0.008,0.093,3.320e-04,0.314,2.750e-05,2.706e-04,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,7.409e-05,0.096,0.253,0.098,0.004,0.028,6.017e-04,-5.769,6.416e-05,0.205,1.759e-04
EX30,0.002,4.713e-05,0.002,0.001,0.067,0.016,5.612e-04,0.042,0.012,0.003,0.017,0.021,0.02,0.003,0.002,7.715e-05,4.469e-04,1.220e-04,3.702e-04,0.003,0.002,8.063e-04,1.247e-04,0.019,6.234e-04,...,0.008,0.017,1.350e-02,0.058,2.750e-05,4.971e-05,8.234e-04,0.066,0.006,0.003,0.317,0.017,1.286e-04,3.621e-04,1.361e-05,0.018,0.047,0.018,0.004,0.028,6.017e-04,0.010,-4.976e+00,0.038,1.759e-04


In [287]:
total_count = 0
for i in range(2013,2024):
    nl_elasticities1 = nl_results.compute_elasticities(market_id=i)
    diagonal = np.diag(pd.DataFrame(nl_elasticities1).values)
    count = np.sum(np.abs(diagonal) < 1)
    total_count += count

print("Total count:", total_count)

Total count: 0


# Cost side

In [288]:
product_data['CCP'] = nl_results.compute_probabilities()
product_data['costs'] = nl_results.compute_costs()
product_data['profit_per_car'] = product_data['prices'] - product_data['costs']
product_data['markups'] = product_data['profit_per_car'] / product_data['costs']
product_data['profits'] = nl_results.compute_profits()
product_data['variable_profits'] = product_data['Sales']*product_data['markups']
product_data[['prices', 'CCP', 'costs', 'profit_per_car', 'markups', 'profits', 'variable_profits']].describe()

,prices,CCP,costs,profit_per_car,markups,profits,variable_profits
count,334.000,3.340e+02,334.000,334.000,334.000,3.340e+02,334.000
mean,46.911,2.128e-03,38.734,8.178,0.346,1.868e-02,137.276
std,29.975,5.618e-03,29.870,1.087,0.319,5.915e-02,408.857
min,12.486,4.347e-06,3.612,7.398,0.040,3.216e-05,0.089
25%,28.398,1.227e-04,20.797,7.478,0.174,9.605e-04,5.652
50%,35.904,7.149e-04,28.034,7.707,0.281,5.795e-03,38.567
75%,52.962,2.161e-03,45.216,8.397,0.373,1.808e-02,153.848
max,194.052,8.814e-02,186.577,11.421,2.457,9.689e-01,6676.021


In [289]:
supply_side = smf.ols('np.log(costs) ~ 0 + Range + HP + Chargetime + C(market_ids) + C(firm_ids)', product_data)
supply_results = supply_side.fit(cov_type='HC3')
supply_results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                      Results: Ordinary least squares
============================================================================
Model:                   OLS                 Adj. R-squared:        0.858   
Dependent Variable:      np.log(costs)       AIC:                   90.9867 
Date:                    2024-06-06 21:05    BIC:                   296.7883
No. Observations:        334                 Log-Likelihood:        8.5067  
Df Model:                53                  F-statistic:           nan     
Df Residuals:            280                 Prob (F-statistic):    nan     
R-squared:               0.881               Scale:                 0.066373
----------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------------
C(market_ids)[2013]           1.7744   0.2216  8.0060 0.0000  1.3400  2.2088
C(market_ids)[2014]           1.8770   0.2769  6.7794 0.0000  1.3343  2.4196
C(market_ids)[2015]           2.0032   0.2377  8.4273 0.0000  1.5373  2.4691
C(market_ids)[2016]           2.1870   0.2170 10.0786 0.0000  1.7617  2.6123
C(market_ids)[2017]           2.2534   0.1848 12.1929 0.0000  1.8912  2.6157
C(market_ids)[2018]           2.2753   0.1838 12.3766 0.0000  1.9150  2.6357
C(market_ids)[2019]           2.2573   0.1789 12.6156 0.0000  1.9066  2.6080
C(market_ids)[2020]           2.2603   0.1756 12.8720 0.0000  1.9161  2.6044
C(market_ids)[2021]           2.2784   0.1749 13.0259 0.0000  1.9356  2.6213
C(market_ids)[2022]           2.3403   0.1750 13.3756 0.0000  1.9974  2.6832
C(market_ids)[2023]           2.3033   0.1777 12.9617 0.0000  1.9550  2.6516
C(firm_ids)[T.Audi]           0.5316   0.1160  4.5845 0.0000  0.3043  0.7589
C(firm_ids)[T.BMW]            0.4110   0.1148  3.5801 0.0003  0.1860  0.6360
C(firm_ids)[T.BYD]           -0.1446   0.1522 -0.9498 0.3422 -0.4430  0.1538
C(firm_ids)[T.Citroen]        0.4020   0.1248  3.2214 0.0013  0.1574  0.6466
C(firm_ids)[T.Cupra]         -0.0249   0.1168 -0.2133 0.8311 -0.2538  0.2040
C(firm_ids)[T.DS]             0.2325   0.1308  1.7782 0.0754 -0.0238  0.4888
C(firm_ids)[T.Dacia]         -0.9161   0.2094 -4.3745 0.0000 -1.3265 -0.5056
C(firm_ids)[T.Fiat]           0.0826   0.1077  0.7675 0.4428 -0.1284  0.2937
C(firm_ids)[T.Fisker]        -0.4904   1.1379 -0.4310 0.6665 -2.7205  1.7398
C(firm_ids)[T.Ford]           0.1459   0.1287  1.1339 0.2568 -0.1063  0.3980
C(firm_ids)[T.Honda]          0.2707   0.1065  2.5406 0.0111  0.0619  0.4795
C(firm_ids)[T.Hongqi]         0.1347   2.0775  0.0649 0.9483 -3.9372  4.2066
C(firm_ids)[T.Hyundai]        0.0810   0.1087  0.7453 0.4561 -0.1320  0.2940
C(firm_ids)[T.JAC]           -0.2591   1.6534 -0.1567 0.8755 -3.4998  2.9815
C(firm_ids)[T.Jaguar]         0.5623   0.1054  5.3325 0.0000  0.3556  0.7689
C(firm_ids)[T.Jeep]           0.2535   0.8056  0.3147 0.7530 -1.3255  1.8325
C(firm_ids)[T.Kia]            0.1257   0.1028  1.2221 0.2217 -0.0759  0.3273
C(firm_ids)[T.Lexus]          0.6464   0.1936  3.3384 0.0008  0.2669  1.0259
C(firm_ids)[T.MG]            -0.0017   0.1203 -0.0143 0.9886 -0.2375  0.2341
C(firm_ids)[T.Maxus]          0.4917   0.0975  5.0443 0.0000  0.3006  0.6827
C(firm_ids)[T.Mazda]          0.0867   0.1235  0.7023 0.4825 -0.1553  0.3287
C(firm_ids)[T.Mercedes-Benz]  0.8169   0.1225  6.6702 0.0000  0.5769  1.0570
C(firm_ids)[T.Mini]           0.3877   0.1782  2.1757 0.0296  0.0384  0.7369
C(firm_ids)[T.Nio]           -0.1683   0.1507 -1.1166 0.2642 -0.4636  0.1271
C(firm_ids)[T.Nissan]         0.2011   0.1019  1.9724 0.0486  0.0013  0.4009
C(firm_ids)[T.Opel]           0.2538   0.1315  1.9302 0.0536 -0.0039  0.5116
C(firm_ids)[T.Peugeot]        0.3723   0.1454  2.5610 0.0104  0.0874  0.6573
C(firm_ids)[T.Polestar]      -0.3193   0.1190 -2.6823 0.0073 -0.5526 -0.0860
C(firm_ids)[T.Porsche]        0.9118   0.1265  